# Parse FASTA Files from MC_fasta Directory

This notebook reads FASTA files from the `mc_info_from_xml/MC_fasta/` folder, extracts protein information, and creates a comprehensive DataFrame with columns: `mcid`, `protein`, `range`,`aa_length`, and `aa_seq`.

## Section 1: Import Required Libraries

In [1]:
# Necessary imports 
import pandas as pd
import os
from pathlib import Path
from typing import List, Tuple, Dict

## Section 2: Set Up File Path and Parameters

In [2]:
# Path to the FASTA folder
fasta_folder = Path('/u/mdmc/enyanduk/internship_areasciencepark/Data/dpcfam/dpcfam_standard/zenodo_unzipped_folders/metaclusters_xml/mc_info_from_xml/MC_fasta/')

# Verify the folder exists
if fasta_folder.exists():
    print(f"Folder found at: {fasta_folder.absolute()}")
else:
    print(f"Folder not found at: {fasta_folder}")

# List all FASTA files in the folder
fasta_files = list(fasta_folder.glob('*.fasta'))
print(f"\nFound {len(fasta_files)} FASTA files")
if fasta_files:
    print("First few files:")
    for f in fasta_files[:5]:
        print(f"  - {f.name}")

Folder found at: /u/mdmc/enyanduk/internship_areasciencepark/Data/dpcfam/dpcfam_standard/zenodo_unzipped_folders/metaclusters_xml/mc_info_from_xml/MC_fasta

Found 46828 FASTA files
First few files:
  - MC82027.fasta
  - MC411219.fasta
  - MC414285.fasta
  - MC450354.fasta
  - MC45741.fasta


## Section 3: Create Function to Parse FASTA Files

In [3]:
def parse_fasta_file(filepath: Path) -> List[Tuple[str, str, str]]:
    """
    Parse a FASTA file and extract protein, range, and amino acid sequence.
    
    Args:
        filepath: Path to the FASTA file
        
    Returns:
        List of tuples containing (protein_id, range, aa_sequence)
    """
    entries = []
    
    with open(filepath, 'r') as f:
        current_protein = None
        current_range = None
        current_sequence = []
        
        for line in f:
            line = line.strip()
            
            if line.startswith('>'):
                # Save the previous entry if it exists
                if current_protein is not None:
                    aa_sequence = ''.join(current_sequence)
                    entries.append((current_protein, current_range, aa_sequence))
                
                # Parse the header line
                # Format: >Protein/Range
                header = line[1:]  # Remove the '>'
                
                if '/' in header:
                    protein_id, range_str = header.split('/', 1)
                    current_protein = protein_id
                    current_range = range_str
                else:
                    # Handle case where there's no range
                    current_protein = header
                    current_range = ''
                
                current_sequence = []
            else:
                # This is a sequence line
                if line:  # Only add non-empty lines
                    current_sequence.append(line)
        
        # Don't forget to add the last entry
        if current_protein is not None:
            aa_sequence = ''.join(current_sequence)
            entries.append((current_protein, current_range, aa_sequence))
    
    return entries

In [4]:
# Test the function on a sample file
if fasta_files:
    sample_file = fasta_files[0]
    print(f"Testing parser on: {sample_file.name}")
    sample_entries = parse_fasta_file(sample_file)
    print(f"Found {len(sample_entries)} entries")
    if sample_entries:
        print(f"\nFirst entry example:")
        protein, range_val, aa = sample_entries[0]
        print(f"  Protein: {protein}")
        print(f"  Range: {range_val}")
        print(f"  AA (first 100 chars): {aa[:100]}...")
        print(f"  Total AA length: {len(aa)}")

Testing parser on: MC82027.fasta
Found 181 entries

First entry example:
  Protein: E8W9U6
  Range: 2-131
  AA (first 100 chars): ILLLSTSDTDLLSARAAQGPVSYRYANPSRLSLDALPQLLEGTDLVVVRLLGGVRAWQEGLDQVLATGRPVVVLTGEQAPDAQLMAASTVPVGIAAEAHA...
  Total AA length: 130


## Section 4: Iterate Through FASTA Files in Directory

In [5]:
# Iterate through all FASTA files and collect data
all_data = []

print(f"Processing {len(fasta_files)} FASTA files...\n")

for idx, fasta_file in enumerate(fasta_files, 1):
    # Extract MCID from filename (e.g., MC1.fasta -> MC1)
    mcid = fasta_file.stem  # stem removes the .fasta extension
    
    # Parse the FASTA file
    try:
        entries = parse_fasta_file(fasta_file)
        
        # Add MCID to each entry
        for protein, range_val, aa_sequence in entries:
            all_data.append({
                'MCID': mcid,
                'Protein': protein,
                'Range': range_val,
                'AA': aa_sequence
            })
        
        print(f"[{idx}/{len(fasta_files)}] {fasta_file.name}: {len(entries)} entries processed")
    
    except Exception as e:
        print(f"[{idx}/{len(fasta_files)}] ✗ Error processing {fasta_file.name}: {str(e)}")

print(f"\nTotal entries collected: {len(all_data)}")

Processing 46828 FASTA files...

[1/46828] MC82027.fasta: 181 entries processed
[2/46828] MC411219.fasta: 316 entries processed
[3/46828] MC414285.fasta: 387 entries processed
[4/46828] MC450354.fasta: 1387 entries processed
[5/46828] MC45741.fasta: 94 entries processed
[6/46828] MC405007.fasta: 78 entries processed
[7/46828] MC413028.fasta: 297 entries processed
[8/46828] MC127398.fasta: 61 entries processed
[9/46828] MC290201.fasta: 207 entries processed
[10/46828] MC53387.fasta: 82 entries processed
[11/46828] MC424352.fasta: 88 entries processed
[12/46828] MC94409.fasta: 128 entries processed
[13/46828] MC94016.fasta: 52 entries processed
[14/46828] MC50490.fasta: 234 entries processed
[15/46828] MC360087.fasta: 73 entries processed
[16/46828] MC64546.fasta: 128 entries processed
[17/46828] MC170064.fasta: 106 entries processed
[18/46828] MC282100.fasta: 109 entries processed
[19/46828] MC39009.fasta: 50 entries processed
[20/46828] MC225843.fasta: 302 entries processed
[21/46828] 

## Section 5: Process and Extract Data

In [6]:
# Add calculated column for sequence length
for entry in all_data:
    entry['Sequence_Length'] = len(entry['AA'])

# Display data statistics
print(f"Total entries: {len(all_data)}")
print(f"Number of unique MCIDs: {len(set(entry['MCID'] for entry in all_data))}")
print(f"Number of unique Proteins: {len(set(entry['Protein'] for entry in all_data))}")
print(f"\nSequence length statistics:")
sequence_lengths = [entry['Sequence_Length'] for entry in all_data]
print(f"  Min: {min(sequence_lengths)}")
print(f"  Max: {max(sequence_lengths)}")
print(f"  Average: {sum(sequence_lengths)/len(sequence_lengths):.2f}")

# Show sample entries
print(f"\nFirst 3 entries:")
for i, entry in enumerate(all_data[:3], 1):
    print(f"  {i}. MCID={entry['MCID']}, Protein={entry['Protein']}, Range={entry['Range']}, "
          f"AA_length={entry['Sequence_Length']}")

Total entries: 15401952
Number of unique MCIDs: 46828
Number of unique Proteins: 10027691

Sequence length statistics:
  Min: 27
  Max: 4034
  Average: 180.35

First 3 entries:
  1. MCID=MC82027, Protein=E8W9U6, Range=2-131, AA_length=130
  2. MCID=MC82027, Protein=I7DYJ9, Range=1-143, AA_length=143
  3. MCID=MC82027, Protein=A0A0M6ZLI4, Range=13-175, AA_length=163


## Section 6: Create DataFrame from Extracted Data

In [7]:
# Create DataFrame with the four required columns
df = pd.DataFrame(all_data)

# Select and reorder columns to match requirements
df = df[['MCID', 'Protein', 'Range', 'AA']]

print("DataFrame created successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumn dtypes:")
print(df.dtypes)
print(f"\nDataFrame info:")
print(df.info())

DataFrame created successfully!
Shape: (15401952, 4)

Column dtypes:
MCID       str
Protein    str
Range      str
AA         str
dtype: object

DataFrame info:
<class 'pandas.DataFrame'>
RangeIndex: 15401952 entries, 0 to 15401951
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   MCID     str  
 1   Protein  str  
 2   Range    str  
 3   AA       str  
dtypes: str(4)
memory usage: 470.0 MB
None


## Section 7: Display and Save Results

In [8]:
# Add a column for sequence length
df['AALength'] = df['AA'].apply(len)
# Rearrange  columns to put AA at the end
df = df[['MCID', 'Protein', 'Range', 'AALength', 'AA']]

In [9]:
# Display the head of the DataFrame
print("\nDataFrame head:")
df.head()


DataFrame head:


,MCID,Protein,Range,AALength,AA
0,MC82027,E8W9U6,2-131,130,ILLLSTSDTDLLSARAAQGPVSYRYANPSRLSLDALPQLLEGTDLV...
1,MC82027,I7DYJ9,1-143,143,MHVVFRESHGLDESDTPQDLGQTPADLVVLSFSDSDLGAFAAGWHR...
2,MC82027,A0A0M6ZLI4,13-175,163,DGGDAVDLGQAPADILFLSAADTELGSFSAAHSALGTDSASLRLAN...
3,MC82027,A0A1X4ITI5,1-182,182,MHLLAATPGNLDDGLEPVDLGQSPADLVVLSAADTDLAALSEARNL...
4,MC82027,A0A1Q7WDG5,5-140,136,FVLLSTADTDLLAARASGAPWRVANPTRLGAADLPALLDGATLAVV...


In [10]:
# Summary statistics
print("\nSummary statistics:")
df.describe().T


Summary statistics:


,count,mean,std,min,25%,50%,75%,max
AALength,15401952.0,180.351937,141.784236,27.0,88.0,139.0,228.0,4034.0


In [11]:
# More derailed summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"Total rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"\nMCIDs represented: {df['MCID'].nunique()}")
print(f"Unique Proteins: {df['Protein'].nunique()}")

print(f"\nMCID distribution:")
print(df['MCID'].value_counts().head(10))

# Check for any missing values
print(f"\nMissing values:")
print(df.isnull().sum())

# Amino acid sequence length statistics
print(f"\nAmino acid sequence length statistics:")
df['AA_Length'] = df['AA'].str.len()
print(f"  Min length: {df['AA_Length'].min()}")
print(f"  Max length: {df['AA_Length'].max()}")
print(f"  Mean length: {df['AA_Length'].mean():.2f}")
print(f"  Median length: {df['AA_Length'].median():.2f}")


SUMMARY STATISTICS
Total rows: 15401952
Total columns: 5

MCIDs represented: 46828
Unique Proteins: 10027691

MCID distribution:
MCID
MC205449    131827
MC85821     126287
MC94243     122226
MC325957    104727
MC427293    101271
MC337317     96176
MC377194     85520
MC238219     78800
MC67266      73185
MC157051     73059
Name: count, dtype: int64

Missing values:
MCID        0
Protein     0
Range       0
AALength    0
AA          0
dtype: int64

Amino acid sequence length statistics:
  Min length: 27
  Max length: 4034
  Mean length: 180.35
  Median length: 139.00


In [12]:
# Info
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 15401952 entries, 0 to 15401951
Data columns (total 6 columns):
 #   Column     Dtype
---  ------     -----
 0   MCID       str  
 1   Protein    str  
 2   Range      str  
 3   AALength   int64
 4   AA         str  
 5   AA_Length  int64
dtypes: int64(2), str(4)
memory usage: 705.0 MB


In [13]:
# Let's organise columns properly for PostgreSQL needs :
# 1. We drop AA_Length:
df = df.drop(columns=["AA_Length"])
# 2. We renamme columns properly:
df = df.rename(columns={
    "MCID":"mcid",
    "Protein":"protein_id",
    "Range":"seq_range",
    "AALength":"seq_length",
    "AA" : "aa_seq"
    })
df.head()

,mcid,protein_id,seq_range,seq_length,aa_seq
0,MC82027,E8W9U6,2-131,130,ILLLSTSDTDLLSARAAQGPVSYRYANPSRLSLDALPQLLEGTDLV...
1,MC82027,I7DYJ9,1-143,143,MHVVFRESHGLDESDTPQDLGQTPADLVVLSFSDSDLGAFAAGWHR...
2,MC82027,A0A0M6ZLI4,13-175,163,DGGDAVDLGQAPADILFLSAADTELGSFSAAHSALGTDSASLRLAN...
3,MC82027,A0A1X4ITI5,1-182,182,MHLLAATPGNLDDGLEPVDLGQSPADLVVLSAADTDLAALSEARNL...
4,MC82027,A0A1Q7WDG5,5-140,136,FVLLSTADTDLLAARASGAPWRVANPTRLGAADLPALLDGATLAVV...


In [14]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 15401952 entries, 0 to 15401951
Data columns (total 5 columns):
 #   Column      Dtype
---  ------      -----
 0   mcid        str  
 1   protein_id  str  
 2   seq_range   str  
 3   seq_length  int64
 4   aa_seq      str  
dtypes: int64(1), str(4)
memory usage: 587.5 MB


In [15]:
# Save the DataFrame to CSV file
output_path = Path('/u/mdmc/enyanduk/internship_areasciencepark/Dataframes/DPCFam/dpcfam_sequences.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(output_path, index=False)
print(f"\nDataFrame saved to: {output_path.absolute()}")
print(f"File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

print("\n" + "="*80)
print("PROCESSING COMPLETE!")
print("="*80)


DataFrame saved to: /u/mdmc/enyanduk/internship_areasciencepark/Dataframes/DPCFam/dpcfam_sequences.csv
File size: 3102.43 MB

PROCESSING COMPLETE!
